In [1]:
! pip install pandas sentence-transformers transformers faiss-cpu streamlit

In [ ]:
import pandas as pd

def load_documents(path="Training Dataset_8.csv"):
    df = pd.read_csv(path)
    df.fillna("N/A", inplace=True)
    documents = df.apply(lambda row: " | ".join(f"{col}: {val}" for col, val in row.items()), axis=1)
    return documents.tolist()

documents = load_documents()
print(f"Loaded {len(documents)} documents")
documents[:2]


Loaded 614 documents


C:\Users\m8cg2\AppData\Local\Temp\ipykernel_44792\3056744019.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("N/A", inplace=True)


['Loan_ID: LP001002 | Gender: Male | Married: No | Dependents: 0 | Education: Graduate | Self_Employed: No | ApplicantIncome: 5849 | CoapplicantIncome: 0.0 | LoanAmount: N/A | Loan_Amount_Term: 360.0 | Credit_History: 1.0 | Property_Area: Urban | Loan_Status: Y',
 'Loan_ID: LP001003 | Gender: Male | Married: Yes | Dependents: 1 | Education: Graduate | Self_Employed: No | ApplicantIncome: 4583 | CoapplicantIncome: 1508.0 | LoanAmount: 128.0 | Loan_Amount_Term: 360.0 | Credit_History: 1.0 | Property_Area: Rural | Loan_Status: N']

In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = model.encode(documents, convert_to_numpy=True, show_progress_bar=True)

# Save for later reuse
np.save("doc_embeddings.npy", doc_embeddings)


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

d:\Anaconda\Lib\site-packages\torch\nn\modules\module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [4]:
from sentence_transformers.util import semantic_search

def retrieve(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    hits = semantic_search(query_embedding, doc_embeddings, top_k=top_k)[0]
    return [documents[hit['corpus_id']] for hit in hits]

# Example
query = "Who are most likely to get loan approved?"
top_chunks = retrieve(query, top_k=3)
for i, chunk in enumerate(top_chunks, 1):
    print(f"Doc {i}:", chunk, "\n")


Doc 1: Loan_ID: LP002447 | Gender: Male | Married: Yes | Dependents: 2 | Education: Not Graduate | Self_Employed: No | ApplicantIncome: 1958 | CoapplicantIncome: 1456.0 | LoanAmount: 60.0 | Loan_Amount_Term: 300.0 | Credit_History: N/A | Property_Area: Urban | Loan_Status: Y 

Doc 2: Loan_ID: LP001091 | Gender: Male | Married: Yes | Dependents: 1 | Education: Graduate | Self_Employed: N/A | ApplicantIncome: 4166 | CoapplicantIncome: 3369.0 | LoanAmount: 201.0 | Loan_Amount_Term: 360.0 | Credit_History: N/A | Property_Area: Urban | Loan_Status: N 

Doc 3: Loan_ID: LP001682 | Gender: Male | Married: Yes | Dependents: 3+ | Education: Not Graduate | Self_Employed: No | ApplicantIncome: 3992 | CoapplicantIncome: 0.0 | LoanAmount: N/A | Loan_Amount_Term: 180.0 | Credit_History: 1.0 | Property_Area: Urban | Loan_Status: N 



In [6]:
! pip install accelerate


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2", trust_remote_code=True)

def answer_question(query, context_docs):
    context = "\n".join(context_docs)
    prompt = f"Use the context to answer:\n{context}\n\nQ: {query}\nA:"
    output = generator(prompt, max_new_tokens=150)[0]['generated_text']
    return output.strip()

# Test
response = answer_question(query, top_chunks)
print("Answer:", response)